In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from pathlib import Path
import os

# 1. 프로젝트 루트 기준 상대경로로 PDF 경로 지정
base_dir = Path(__file__).parent  # 현재 코드 파일 기준 폴더
pdf_paths = [
    base_dir / "pdf" / "2023_핵심_컴퓨터활용능력1급_필기.pdf"
]

# PDF 파일 존재 여부 체크
for path in pdf_paths:
    if not path.exists():
        raise FileNotFoundError(f"PDF 파일이 없습니다: {path}")

# 2. PDF 문서 로드
all_docs = []
for path in pdf_paths:
    loader = PyPDFLoader(str(path))
    docs = loader.load()
    all_docs.extend(docs)

# 3. PDF 문서를 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)
split_docs = text_splitter.split_documents(all_docs)

# 4. 단어 임베딩 생성
openai_api_key = "본인의-OpenAI-API-키-여기에-입력"
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# 5. 벡터스토어에 청크 저장
vectorstore = FAISS.from_documents(split_docs, embeddings)

# 6. 벡터스토어 저장 폴더 생성 (상대경로)
save_dir = base_dir / "faiss_db"
save_dir.mkdir(parents=True, exist_ok=True)

# 7. 벡터스토어 로컬 저장
vectorstore.save_local(folder_path=str(save_dir), index_name="index")

print(f"✅ 벡터스토어 저장 완료: {save_dir.resolve()}")
